In [1]:
# libraries
import numpy as np
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import cv2

In [2]:
# converting color image into gray image
def grayscale(image):
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    return gray

In [3]:
# applying gaussian-blur
def gaussian_blur(image, kernel_size = 5):
    blur_gray = cv2.GaussianBlur(image,(kernel_size, kernel_size),0)
    return blur_gray

In [4]:
# applying canny edge detection
def canny(image, low_threshold=50, high_threshold=150):
    edges = cv2.Canny(image, low_threshold, high_threshold)
    return edges

In [5]:
# getting region of interest from image
def region_of_interest(edges):
    # creating a mask
    mask = np.zeros_like(edges)
    ignore_mask_color = 255
    imshape = edges.shape[:2]
    # four sided polygon
    vertices = np.array([[(150,imshape[0]),(460, 320), (540, 320), (860, imshape[0])]], dtype = np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    # masking edges
    masked_edges = cv2.bitwise_and(edges, mask)
    return masked_edges

In [6]:
# applying Hough transform
def hough_lines(masked_edges):
    # parameters for hough
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 20    # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #minimum number of pixels making up a line
    max_line_gap = 300    # maximum gap in pixels between connectable line segments
    # applying hough
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
    return lines

In [7]:
# interpolating and drawing lane lines
def draw_lines(image, lines):
    left_lines = [] # in form of (slope, intercept)
    left_weights = [] # have 'length'
    right_lines = [] # in form of (slope, intercept)
    right_weights = [] # have 'length'
    for line in lines:
        for x1, y1, x2, y2 in line:
            # vertical lines have infinite slope, so ignore them
            if x2==x1:
                continue 
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            # negative slope implies left lane else right lane
            if slope < 0: 
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    # elongating longer lines
    left_lane  = np.dot(left_weights,  left_lines) /np.sum(left_weights)  if len(left_weights) >0 else None
    right_lane = np.dot(right_weights, right_lines)/np.sum(right_weights) if len(right_weights)>0 else None
    # mask to draw lines
    line_image = np.copy(image)*0
    # converting lines into point coordinates
    y1 = image.shape[0]
    y2 = y1 * 0.6
    left_line = make_line_points(y1, y2, left_lane)
    right_line = make_line_points(y1, y2, right_lane)
    if(left_line is not None):
        cv2.line(line_image,(left_line[0][0],left_line[0][1]),(left_line[1][0],left_line[1][1]),(255,0,0),10)
    if(right_line is not None):
        cv2.line(line_image,(right_line[0][0],right_line[0][1]),(right_line[1][0],right_line[1][1]),(255,0,0),10)
    return line_image

In [8]:
# weighing marked lanes on image
def weighted_image(image, edges, line_image):
    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges))
    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(image, 1.0, line_image, 1, 0)
    return lines_edges

In [9]:
# convert a line represented in slope and intercept into pixel points
def make_line_points(y1, y2, line):
    if line is None:
        return None
    slope, intercept = line
    # make sure everything is integer as cv2.line requires it
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))

In [10]:
# main function
def main_process(image):
    # getting gray image
    gray = grayscale(image)
    # applying gaussian blur
    blur_gray = gaussian_blur(gray)
    # applying canny edge detection
    edges = canny(blur_gray)
    # applying region of interest
    masked_edges = region_of_interest(edges)
    # applying hough
    lines = hough_lines(masked_edges)
    # drawing lines on image
    line_image = draw_lines(image, lines)
    # weighing marked lanes on image
    result = weighted_image(image, edges, line_image)
    return result

In [11]:
# reading video clip
clip = VideoFileClip("./test_videos/solidYellowLeft.mp4")
# processing frames of video by passing them in "main_process"
processed = clip.fl_image(main_process)
# saving output
processed.write_videofile("./output_solidYellowLeft.mp4", audio=False)
# reading second video clip
clip = VideoFileClip("./test_videos/solidWhiteRight.mp4")
# processing frames of video by passing them in "main_process"
processed = clip.fl_image(main_process)
# saving output
processed.write_videofile("./output_solidWhiteRight.mp4", audio=False)

[MoviePy] >>>> Building video ./output_solidYellowLeft.mp4
[MoviePy] Writing video ./output_solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:23<00:00, 28.69it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_solidYellowLeft.mp4 

[MoviePy] >>>> Building video ./output_solidWhiteRight.mp4
[MoviePy] Writing video ./output_solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:07<00:00, 30.15it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_solidWhiteRight.mp4 

